In [1]:
import json
import sys
import numpy as np
sys.path.append('../library')
from clustering import *
from helpers import *
import scipy

In [2]:
def get_values(X):
    X_list = []
    for elem in X:
        X_list.append(X[elem])
    return X_list

def get_keys(X):
    X_list = []
    for elem in X:
        X_list.append(elem)
    return X_list

###
### Transforma un obiect fcm [[elems_clasa_0],..,[elem_clasa_n]] in lista de labels [class_elem_0, .., class_elem_n].
### param labels: obiect de tip fcm.
### return lista.
###
def create_class_labels(labels,venues, size):
    l = np.zeros(size, dtype=int)
    i = 0
    for class_ in labels:
        for elem in class_:
            l[elem] = i
        i += 1
    dict_ = {}
    for i,elem in enumerate(venues):
        dict_[elem] = int(l[i])
    return dict_

def run_fuzzy(path, cls, camp = 'd2w', print_ = False):
    X = {}
    r = open(path, 'r')
    for line in r:
        art_crt = json.loads(line)
        X[art_crt['id']] = art_crt[camp]
    r.close()
    
    X_values = get_values(X)
    X_keys = get_keys(X)
    
    if print_ == True:
        print('size vector ',len(X), len(X_values))
    fcm = apply_fuzzy(X_values,cls)
    labels = fcm.get_membership()
    
    i = 0
    X_dict = {}
    for elem in labels:
        X_dict[X_keys[i]] = elem 
        i+=1
    return X_dict, cls
    

###
### Transforma lista de weights a clusterelor in dictionar de forma {cluster:weight}, daca weight este mai 
### mare decat mijlocul de valori din weights initial.
### param weights: lista de weights initiala.
### return dict_ sau -1 daca nu este nici o valoare din lista  mai maire de valoarea de middle.
###
def get_class_weights(weights):
    dict_ = {}
    min_ = min(weights)
    max_ = max(weights)
    middle = (max_ + min_)/2
    for i in range(0,len(weights)):
        if weights[i] >= middle:
            dict_[i] = weights[i]
            
    if len(dict_) == 0:
        return -1
    return dict_


    
def print_results(mem, path_in, path_out, nrFiles, space= False):
    excluded = 0
    for i in range(0, nrFiles):
        r = open(path_in+str(i)+'.txt', 'r')
        w = open(path_out+str(i)+'.txt', 'w')
        
        for line in r:
            art_crt = json.loads(line)
            new_art = {}
            new_art['id'] = art_crt['id']
            new_art['venue'] = art_crt['venue']
            new_art['abstract'] = art_crt['abstract']
            new_art['fos'] = art_crt['fos']
            try:
                new_art['class_weights'] = get_class_weights(mem[new_art['id']])
            except KeyError:
                excluded += 1
            
            w.write(json.dumps(new_art))
            w.write('\n')
            
        w.close()
        r.close()
    return excluded

## Complete

## Doc2Vec

In [12]:
mem, cls = run_fuzzy('../Date/doc2vec/Abstract/Subset/file0.txt', 10)
print(len(mem), cls)

10000 10


In [31]:
for elem in mem_max:
    print (elem, mem_max[elem])
    break

001c58d3-26ad-46b3-ab3a-c1e557d16821 [0.49999815964999705, 0.5000018403500031]


In [13]:
print_results(mem, '../Date/Initiale/Subset/file', '../Rezultate/Fuzzy/Abstract/doc2vec/10clus_file', 1)

0

## Word2Vec pretrained

### Avg

In [76]:
mem,cls = run_fuzzy('../Date/word2vec-pretrained/Abstract/Subset/avg/file0.txt', 16,'abstract')
print(len(mem), cls)

10000 16


In [79]:
excluded = print_results(mem, '../Date/Initiale/Subset/file', '../Rezultate/Fuzzy/Abstract/word2vec-pretrained/avg/16clus_file', 1, True)
print(excluded)

10000


### Sum

In [80]:
mem,cls = run_fuzzy('../Date/word2vec-pretrained/Abstract/Subset/sum/file0.txt', 18, 'abstract')

In [86]:
excluded = print_results(mem, '../Date/Initiale/Complet/file', '../Rezultate/Fuzzy/Abstract/word2vec-pretrained/sum/18clus_file', 1, True)
print(excluded)

0


## Word2vec gensim

### Avg

In [84]:
mem,cls = run_fuzzy('../Date/word2vec-gensim/Abstract/Subset/avg/file0.txt', 24, 'abstract')
print(len(mem), cls)

10000 24


In [85]:
excluded = print_results(mem, '../Date/Initiale/Complet/file', '../Rezultate/Fuzzy/Abstract/word2vec-gensim/avg/24clus_file', 1, True)
print(excluded)

0


### Sum

In [88]:
mem,cls = run_fuzzy('../Date/word2vec-gensim/Abstract/Subset/sum/file0.txt', 20, 'abstract')
print(len(mem), cls)

10000 20


In [90]:
excluded = print_results(mem, '../Date/Initiale/Complet/file', '../Rezultate/Fuzzy/Abstract/word2vec-gensim/sum/20clus_file', 1, True)
print(excluded)

0


## Evaluate fuzzy

In [3]:
###
### Verifica daca o cheie se gaseste intr-un dictionar si returneaza valoarea din dictionar.
### param dict_cluster: dictionarul.
### param nr_cluster: cheia
### return True, valoare daca gaseste sau False, -1 daca nu gaseste
###
def is_in_cluster(dict_clusters, nr_cluster):
    try :
        return True, dict_clusters[nr_cluster]
    except KeyError:
        return False, -1

#---
###
### Parcurge fisierele cu weight_cluster si creeaza un dictionar de forma {venue:weight} pentru un cluster dat.
### param cluster: numarul clusterului pentru care se face cautarea (! In format string)
### param path: calea spre fisiere
### param nrFiles: numarul de fisiere
### return cluster_dict: dictionarul cuu venues si weight asociat pentru clusterul cerut
###
def extragere_puncte_cluster(cluster, path, nrFiles):
    cluster_dict = {}
    for i in range(0, nrFiles):
        file = path+str(i)+'.txt'
        r = open(file,'r')
        for line in r:
            
            art_crt = json.loads(line)
            try:
                includes, weight = is_in_cluster(art_crt['class_weights'],cluster) 
            except KeyError:
                continue
            if (includes == True):
                id_ = art_crt['id']
                try :
                    cluster_dict[id_] += weight 
                except KeyError:
                    cluster_dict[id_] = weight   
        r.close()
    return cluster_dict


#--
def calcul_inter_cluster(model, cluster_dict): 
    # am un dict {venue: weight}
    # am modelul care face corespondenta venue -> embedding
    sum_total = 0
    print(len(cluster_dict))
    for elem_crt in cluster_dict:
        for elem in cluster_dict:
            if elem_crt == elem:
                continue
            sum_ = calculeaza_distanta(model[elem], model[elem_crt])
            sum_ *= cluster_dict[elem]
        sum_total += sum_
        
    return sum_total / len(cluster_dict)

#--
## cluster_dict : {venue:weight}
def calc_centro(model, cluster_dict, dim): 
    sum_total = 0
    
    centroid = []
    for i in range(0, dim):
        centroid.append(0)
    
    for elem in cluster_dict:
        elem_m = model[elem]
        for i in range(0, dim):
            centroid[i] += elem_m[i]
            
    for i in range(0, dim):
        centroid[i] /= dim
    return centroid

# -- 
def calcul_intra_cluster(centroid_list):
    dist_total = 0
    
    for centroid_crt in centroid_list:
        dist = 0
        for centroid in centroid_list:
            if centroid == centroid_list:
                continue
            dist += calculeaza_distanta(centroid_crt, centroid)
        dist_total += (dist/(len(centroid_list)-1))
    print('CENTRO', dist)
    return dist / len(centroid_list)
#---
def process(nrClustere, cls, path, nrFiles, size_model):
    total = 0
    centroid_list = []
    for i in range(0,nrClustere):
        cluster_dict = extragere_puncte_cluster(cls[i],path, nrFiles)
        total += calcul_inter_cluster(X_dict, cluster_dict)
        centroid_list.append(calc_centro(X_dict, cluster_dict, size_model))
        
    intra = calcul_intra_cluster(centroid_list)
    total /= nrClustere    
    return total, intra    

## citeste modelul si elimina spatiul final din chei
def read_model(path):
    r = open(path, 'r')
    X_dict = json.load(r)
    r.close()
    new_x_dict = {}
    for elem in X_dict:
        new_elem = elem[0:len(elem)-1]
        new_x_dict[new_elem] = X_dict[elem]
    return new_x_dict

In [4]:
X_dict = {}
r = open('../Date/doc2vec/Abstract/Subset/file0.txt', 'r')
for line in r:
    art_crt = json.loads(line)
    X_dict[art_crt['id']] = art_crt['d2w']
r.close()

## Doc2vec

In [14]:
cls = ['0','1','2','3','4','5','6','7','8','9']

total, intra = process(10,cls,'../Rezultate/Fuzzy/Abstract/doc2vec/10clus_file', 1, 300)
#                                  Rezultate\Fuzzy\Abstract\doc2vec
print(total, intra)

5075
5126
4808
5127
4842
4894
4870
4768
5086
5209
CENTRO 119.52463538168058
0.9638868020669757 11.952463538168058


## Word2Vec pretrained avg

In [ ]:
cls = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15']

total, intra = process(16,cls,'../Rezultate/Fuzzy/Abstract/word2vec-pretrained/avg/16clus_file', 1, 50)
print(total, intra)

4947


## Word2Vec pretrained sum

In [5]:
cls = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']

total, intra = process(18,cls,'../Rezultate/Fuzzy/Abstract/word2vec-pretrained/sum/18clus_file', 1, 50)
print(total, intra) # THIS!!!!!!!! 

5185
4814
5240
5223
4812
4886
4824
5198
5204
4823
4852
4825
4814
5204
4889
5184
5184
5186
CENTRO 582.1423763954053
0.4121444496119219 32.34124313307807


## Word2Vec gensim avg

In [ ]:
cls = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']

total, intra = process(24,cls,'../Rezultate/Fuzzy/Abstract/word2vec-gensim/avg/24clus_file', 1, 50)
print(total, intra)

## Word2Vec gensim sum

In [ ]:
X_dict = read_model('../Date/word2vec-gensim/Venue/w2v_gensim_sum_50_c.txt')
cls = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19']

total, intra = process(20,cls,'../Rezultate/Fuzzy/Abstract/word2vec-gensim/sum/20clus_file', 1, 50)
print(total, intra)